<h1>SRGAN-Modified</h1>

**Index:**

1. Imports
2. Hyperparameters
3. Helper functions
4. Augmentation
5. Dataset and Dataloader
6. Networks
7. Generator and Discriminator
8. Training Loop
9. Statistics


Imports:


In [ ]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import torch.nn as nn
from torchvision.models import vgg19, VGG19_Weights
import torch
from torch import optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

import random

Hyperparameters:


In [ ]:
device = 'cuda' if torch.cuda.is_available else 'cpu'
disc_loss_history = []
gen_loss_history = []

lr = 1e-4
epochs = 7
batch_size = 5
drop = 0.1
low_res_size = 128
high_res_size = 256

snapshot = 2
record = 25

doAugment = False #I recommend keeping this False since the dataset already has 6000 images...
useBottlenecked = True
useDropout = True
useSharpener = False
freezeSharpener = True
pretrained = True

pretrained_params_file = "SRGANParams/SRGAN_M_5.pth"
train_folder = 'SRGANDataMore/train/'
val_folder = 'SRGANDataMore/val/'
params_save_folder = './SRGANParams/'
sharpener_params = ''

Helper Functions:


In [ ]:
def augment_images(start, end, orig_size, transform_type):
    for i in range(start, end):
        for res_quality in ['low_res', 'high_res']:
            im = Image.open(f'{train_folder}{res_quality}/{i}.png')
            im_t = im.transpose(transform_type)
            im_t.save(f"{train_folder}{res_quality}/{orig_size+i}.png")

def upscale(model, image_path, sharpener=None, return_image=False, useSharpener=False):
    model = model.to('cpu').eval()

    transform_low2 = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
    ])
    img  = transform_low2(Image.open(image_path))
    img = img[:3,:,:]
    img = torch.reshape(img, (1,3,128,128))

    upscaled_img = model(img).detach()
    if useSharpener:
        if sharpener is None:
            print("Error: Using sharpener but no sharpener inputted...")
            return
        sharpener = sharpener.to('cpu').eval()
        upscaled_img = sharpener(upscaled_img).detach()
    
    _, axs = plt.subplots(1, 2, figsize=(8,4))
    img = img.permute(0,2,3,1)[0]

    axs[0].set_axis_off()
    axs[0].imshow(img)
    axs[0].set_title("Original")

    axs[1].set_axis_off()
    axs[1].imshow(upscaled_img.permute(0, 2, 3, 1)[0])
    axs[1].set_title("Upscaled")

    plt.show()

    model = model.to(device).train()
    if useSharpener:
        sharpener = sharpener.to('cpu').train()

    if return_image:
        x = 255*(upscaled_img[0].permute(1,2,0)) #[C,H,W]->[H,W,C]
        x = x.numpy().astype(np.uint8)
        return Image.fromarray(x) 
    

transform_low = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((low_res_size, low_res_size)),
    transforms.ToTensor(),
])

transform_high = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((high_res_size, high_res_size)),
    transforms.ToTensor(),
])

def GetLayer(nameOfLayer, *num_params):
    if nameOfLayer=='batch':
        return nn.BatchNorm2d(*num_params)
    elif nameOfLayer=='instance':
        return nn.InstanceNorm2d(*num_params)    
    elif nameOfLayer=='relu':
        return nn.ReLU(False)
    elif nameOfLayer=='lrelu':
        return nn.LeakyReLU(0.2, True)
    elif nameOfLayer=='prelu':
        return nn.PReLU(*num_params)
    elif nameOfLayer=='tanh':
        return nn.Tanh()
    elif nameOfLayer=='sigmoid':
        return nn.Sigmoid()
    else:
        return nn.Identity()

def Checkpoint(model, save_folder, curr_epoch, loss, file_name="", paramLogFilename=""):
    torch.save(model.state_dict(), save_folder+file_name+f"{curr_epoch}.pth")
    print(f"Checkpoint saved to {file_name}{curr_epoch}.pth")
    log = open(save_folder+paramLogFilename, 'a')
    log.write(f"Version:{file_name}{curr_epoch}.pth, Epoch:{curr_epoch}/{epochs}, lr:{lr}, batch_size:{batch_size}, useDropout:{useDropout}, droprate:{drop}, pretrained:{pretrained}, \
              pretrained_params:{pretrained_params_file if pretrained else None}, useSharpener:{useSharpener}, sharpener:{sharpener_params} useBottlenecked:{useBottlenecked}, freezeSharpener:{freezeSharpener}, gen_loss:{loss}\n")
    log.close()
    print(f"Checkpoint details updated to {save_folder+paramLogFilename}")

def PrintNetwork(net):
    num_params = 0
    for param in net.parameters():
        num_params += param.numel()
    print(net)
    print(f'Total number of parameters: {num_params}')

Augmentation: (Run from here if you want to restart training)


In [ ]:
if doAugment:
    augment_images(0, 171, 685, Image.Transpose.ROTATE_180)
    augment_images(171, 342, 685, Image.Transpose.FLIP_TOP_BOTTOM)
    augment_images(342, 514, 685, Image.Transpose.FLIP_LEFT_RIGHT)
    augment_images(514, 685, 685, Image.Transpose.TRANSVERSE)
    doAugment=False

Dataset and Dataloader:


In [ ]:
class ImageDataset(Dataset):
    def __init__(self, root_dir):
        super(ImageDataset, self).__init__()
        self.root_dir = root_dir
        files_low = os.listdir(os.path.join(root_dir, "low_res"))
        files_high = os.listdir(os.path.join(root_dir, "high_res"))
        self.data = list(zip(files_low, files_high))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        #Stores file name of the png
        img_low_file, img_high_file = self.data[index]

        #Gets their paths w.r.t current working directory
        low_res_pth = os.path.join(self.root_dir, "low_res", img_low_file)
        high_res_pth = os.path.join(self.root_dir, "high_res", img_high_file)

        low_res = np.array(Image.open(low_res_pth).convert('RGBA'))
        high_res = np.array(Image.open(high_res_pth).convert('RGBA'))

        low_res = low_res[:, :, :3]
        high_res = high_res[:, :, :3]

        low_res = transform_low(low_res)
        high_res = transform_high(high_res)

        return low_res, high_res
    
train = ImageDataset(root_dir=train_folder)
train_loader = DataLoader(train, batch_size=batch_size, num_workers=2, shuffle=True)

val = ImageDataset(root_dir=val_folder)
val_loader = DataLoader(val, batch_size=batch_size, num_workers=2, shuffle=True)

In [ ]:
def plot_examples(gen):
    dataset_test = ImageDataset(root_dir=val_folder)
    loader = DataLoader(dataset_test, batch_size=16)
    gen.eval()

    # Create a figure with two subplots
    fig, axs = plt.subplots(1, 3, figsize=(8, 4))
    chosen_batch = random.randint(0, len(loader)-1)
    for idx, (low_res, high_res) in enumerate(loader):
        if(chosen_batch == idx):
            chosen = random.randint(0, len(low_res)-1)

            axs[0].set_axis_off()
            axs[0].imshow(low_res[chosen].permute(1, 2, 0))
            axs[0].set_title("Input")

            with torch.no_grad():
                upscaled_img = gen(low_res[chosen].to(device).unsqueeze(0))

            axs[1].set_axis_off()
            axs[1].imshow(upscaled_img.permute(0, 2, 3, 1)[0].cpu())
            axs[1].set_title("Predicted")

            axs[2].set_axis_off()
            axs[2].imshow(high_res[chosen].permute(1, 2, 0))
            axs[2].set_title("Ground Truth")

            if(idx == 1):
                break

    plt.show()

    gen.train()

Networks:


In [ ]:
class vggL(nn.Module):
    def __init__(self):
        super().__init__()
        self.vgg = vgg19(weights=VGG19_Weights.DEFAULT).features[:25].eval().to(device)
        self.MSE = nn.MSELoss()

    def forward(self, first, second):
        vgg_first = self.vgg(first)
        vgg_second = self.vgg(second)
        perceptual_loss = self.MSE(vgg_first, vgg_second)
        return perceptual_loss

class ConvBlock(nn.Module):
    def __init__(
            self,
            in_channels,
            out_channels,
            use_activation=True,
            use_BatchNorm=True,
            **kwargs
    ):

        super(ConvBlock, self).__init__()
        self.use_activation = use_activation
        self.cnn = nn.Conv2d(in_channels, out_channels, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels) if use_BatchNorm else nn.Identity()
        self.ac = nn.LeakyReLU(0.2) 
                                    
    def forward(self, x):
        x1 = self.cnn(x)
        x2 = self.bn(x1)
        x3 = self.ac(x2)
        return x3 if self.use_activation else x2


class UpsampleBlock(nn.Module):
    def __init__(self, in_channels, scale_factor):
        super(UpsampleBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, in_channels * scale_factor ** 2, kernel_size=2, stride=1, padding=1) 
        self.ps = nn.PixelShuffle(scale_factor)
        self.ac = nn.PReLU(num_parameters=in_channels)

    def forward(self, x):
        x = self.conv(x)
        x = self.ps(x)
        return self.ac(x)


class ResidualBlock(nn.Module):
    def __init__(self, in_channels, useDrop=False, useBottlenecked=False):
        super(ResidualBlock, self).__init__()
        self.dropout = nn.Dropout(p=drop) if useDrop else nn.Identity()
        self.bottleneck_in = nn.Identity()
        self.bottleneck_out = nn.Identity()

        if useBottlenecked:
            orig_filter = in_channels
            in_channels = in_channels//2
            self.bottleneck_in = nn.Conv2d(orig_filter, in_channels, kernel_size=3, stride=1, padding=1, bias=True)
            self.bottleneck_out = nn.Conv2d(in_channels, orig_filter, kernel_size=3, stride=1, padding=1, bias=True)

        self.b1 = ConvBlock(
            in_channels,
            in_channels,
            kernel_size=3,
            stride=1,
            padding=1
        )

        self.b2 = ConvBlock(
            in_channels,
            in_channels,
            kernel_size=3,
            stride=1,
            padding=1,
            use_activation=False
        )

    def forward(self, x):
        out = self.bottleneck_in(x)
        out = self.b1(out)
        out = self.b2(out)
        out = self.bottleneck_out(out)
        out = self.dropout(out)
        return out + x 
    
class ConvBlockEnhancer(nn.Module):
    def __init__(self, input_size, output_size, kernel_size=7, stride=1, padding=3, bias=False, activation='prelu', norm=None):
        super(ConvBlockEnhancer, self).__init__()

        self.conv = nn.Conv2d(input_size, output_size, kernel_size=kernel_size, stride=stride, padding=padding, bias=bias)
        self.bn = GetLayer(norm, output_size)
        self.act = GetLayer(activation, output_size)

    def forward(self, x):
        out = self.bn(self.conv(x))
        return self.act(out)
    
class Deblurrer(nn.Module):
    def __init__(self,  kernel_size, channels=3, bias=True, activation='lrelu', norm=None):
        super(Deblurrer, self).__init__()
        stride=1
        padding = (kernel_size-1)//2
        self.conv1 = ConvBlockEnhancer(channels, channels, kernel_size, stride, padding, bias, activation, norm)
        self.conv2 = ConvBlockEnhancer(channels, channels, kernel_size, stride, padding, bias, activation, norm)
        self.conv3 = ConvBlockEnhancer(channels, channels, kernel_size, stride, padding, bias, activation, norm)
        self.conv4 = ConvBlockEnhancer(channels, channels, kernel_size, stride, padding, bias, activation, norm)
        self.conv5 = ConvBlockEnhancer(channels, channels, kernel_size, stride, padding, bias, activation, norm)
        self.conv6 = ConvBlockEnhancer(channels, channels, kernel_size, stride, padding, bias, activation, norm)
        self.dropout = nn.Dropout(p=drop)
        self.act = GetLayer('relu')
    
    def forward(self, x):
        residue1 = x
        x = self.conv1(x)
        x = self.conv2(x)
        x = residue1 + x
        x = self.conv3(x)
        x = self.dropout(x)
        residue2 = x
        x = self.conv4(x)
        x = self.conv5(x)
        x = residue2 + x
        x = self.conv6(x)
        x = residue1 + x
        return self.act(x)

Generator and Discriminator:


In [ ]:
class Generator(nn.Module):
    def __init__(self, in_channels=3, num_channels=64, num_blocks=8, useSharpener=False, freezeSharpener=True, useDropout=False, useBottlenecked=False, pathToSharpener=None):
        super(Generator, self).__init__()
        self.initial = ConvBlock(in_channels, num_channels, kernel_size=7, stride=1, padding=4, use_BatchNorm=False) #K=9 in the paper
        self.res = nn.Sequential(*[ResidualBlock(num_channels, useDrop=useDropout, useBottlenecked=useBottlenecked) for i in range(num_blocks)])
        self.conv = ConvBlock(num_channels, num_channels, kernel_size=3, stride=1, padding=1, use_activation=False)
        self.up = UpsampleBlock(num_channels, scale_factor=2) 
        self.final = nn.Conv2d(num_channels, in_channels, kernel_size=9, stride=1, padding=1)

        self.bicubic_up = transforms.Resize((256, 256), interpolation=transforms.InterpolationMode.BICUBIC)
        self.unblur = nn.Identity()

        if useSharpener:
            if pathToSharpener is None:
                print("Error: Using sharpener with no sharpener loaded...")
            else:
                self.unblur = Deblurrer(kernel_size=5).to(device)
                self.unblur.load_state_dict(torch.load(pathToSharpener, map_location=device))
                if freezeSharpener:
                    for param in self.unblur.parameters():
                        param.requires_grad = False

    def forward(self, x):
        residue = x
        x = self.initial(x)
        c = self.res(x)
        c = self.conv(c) + x 
        c = self.up(c)
        c = self.final(c)
        c = c + self.bicubic_up(residue)
        c = self.unblur(c)
        return torch.sigmoid(c) #Sigmoid activation not mentioned in the paper


class Discriminator(nn.Module):
    def __init__(self, in_channels=3, features=[64, 64, 128, 128, 256, 256, 512, 512]):
        super(Discriminator, self).__init__()
        blocks = []
        for idx, feature in enumerate(features):
            blocks.append(
                ConvBlock(
                    in_channels,
                    feature, #Numbers as given in the paper. See the diagram labels
                    kernel_size=3,
                    stride = idx % 2 + 1, #Even idx gives s=1, Odd idx gives 2. This is as per the architecture proposed in the paper
                    padding=1,
                    use_activation=True,
                    use_BatchNorm=idx != 0 #Only the first block in this has no Batch Normalization
                )
            )
            in_channels = feature  

        self.blocks = nn.Sequential(*blocks)

        self.mlp = nn.Sequential(
            nn.AdaptiveAvgPool2d((8, 8)), #Not there in the paper. Converts input (N,512,H,W) tensor into a (512,C,8,8) tensor
            nn.Flatten(),
            nn.Linear(512*8*8, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 1)
        )

    def forward(self, x):
        x = self.blocks(x)
        x = self.mlp(x)
        return x

In [ ]:
gen = Generator(in_channels=3, useBottlenecked=True, useDropout=True, pathToSharpener=sharpener_params).to(device) #You have access to 4 boolean arguments that can drastically alter the result of the network. Hover over Generator to see them. Default paramaters indicate original SRGAN design
disc = Discriminator(in_channels=3).to(device)

opt_gen = optim.Adam(gen.parameters(), lr=lr, betas=(0.9, 0.999))
opt_disc = optim.Adam(disc.parameters(), lr=lr, betas=(0.9, 0.999))

bce = nn.BCEWithLogitsLoss()
vgg_loss = vggL()

Training Loop:


In [ ]:
def train_fn(loader, disc, gen, opt_gen, opt_disc, bce, vgg_loss):
    disc_loss = 0
    gen_loss = 0

    for idx, (low_res, high_res) in enumerate(loader):
        high_res = high_res.to(device)
        low_res = low_res.to(device)

        fake = gen(low_res)

        disc_real = disc(high_res)
        disc_fake = disc(fake.detach())

        disc_loss_real = bce(disc_real, torch.ones_like(disc_real)) 
        disc_loss_fake = bce(disc_fake, torch.zeros_like(disc_fake)) 

        disc_loss = disc_loss_fake + disc_loss_real

        opt_disc.zero_grad()
        disc_loss.backward()
        opt_disc.step()

        disc_fake = disc(fake)
        adversarial_loss = 1e-3 * bce(disc_fake, torch.ones_like(disc_fake)) 
        loss_for_vgg = 0.006*vgg_loss(fake, high_res) #0.006 factor not mentioned in paper
        gen_loss = loss_for_vgg + adversarial_loss

        opt_gen.zero_grad()
        gen_loss.backward()
        opt_gen.step()

        if (idx+1)%record == 0:
          with torch.no_grad():
            disc_loss_history.append(disc_loss)
            gen_loss_history.append(gen_loss)

    return gen_loss.detach().cpu(), disc_loss.detach().cpu()

In [ ]:
if pretrained:
    if pretrained_params_file=="" or pretrained_params_file is None:
        print("File path to pretrained parameters is empty...")
    else:
        gen.load_state_dict(torch.load(pretrained_params_file, map_location=device))
        print("Parameters successfully loaded to model")

In [ ]:
for epoch in range(epochs):
     gen_loss, disc_loss = train_fn(train_loader, disc, gen, opt_gen, opt_disc, bce, vgg_loss)
     plot_examples(gen)
     print("epoch ", epoch+1, "/", epochs)
     print(f"gen_loss: {gen_loss:.4f}, disc_loss:{disc_loss:.4f}")
     if (epoch+1)%snapshot==0:
          Checkpoint(gen, save_folder=params_save_folder, curr_epoch=epoch+1, loss=gen_loss, file_name="SRGAN_M_", paramLogFilename="SRGAN_M_Paramlog.txt")

Statistics:


In [ ]:
plt.plot([x for x in range(len(gen_loss_history))], [x.detach().cpu() for x in gen_loss_history])

In [ ]:
upscaled_img = upscale(gen, "blur.png", return_image=True)
print(upscaled_img)

In [ ]:
PrintNetwork(gen)
PrintNetwork(disc)